In [1]:
import pandas as pd
import pytz
import MetaTrader5 as mt5
from decouple import config
from datetime import datetime

# Define constants
TIMEZONE = pytz.timezone("Etc/UTC")
TIME_FRAME = mt5.TIMEFRAME_H1
DATE_FROM = datetime(2023, 3, 10, tzinfo=TIMEZONE)
DATE_TO = datetime(2024, 3, 10, tzinfo=TIMEZONE)

# Load configuration
LOGIN = config('LOGIN', cast=int, default=5032967129)
PASSWORD = config('PASSWORD', default="-jEs3jSi")
SERVER = config('SERVER', default="MetaQuotes-Demo")

# Initialize MetaTrader5
if not mt5.initialize(login=LOGIN, password=PASSWORD, server=SERVER):
    print(f"Failed to initialize MetaTrader5, error code: {mt5.last_error()}")
    mt5.shutdown()
    exit()

# Fetch rates data
rates = mt5.copy_rates_range('EURUSD', TIME_FRAME, DATE_FROM, DATE_TO)

# Process data into a DataFrame
df = pd.DataFrame(rates)
df = df.drop(columns=["tick_volume", "spread", "real_volume"])
df['time'] = pd.to_datetime(df['time'], unit='s')
df['time'] = df['time'].dt.strftime('%d.%m.%Y %H:%M:%S.%f')

In [2]:
def pivotid(df1, index, n):
    """Find pivot point with n neighbors before and after."""
    
    # Ensure the index is within a valid range
    if index - n < 0 or index + n >= len(df1):
        return 0
    
    is_low_pivot = True
    is_high_pivot = True
    
    # Check the neighborhood for low and high pivots
    for i in range(index - n, index + n + 1):
        if df1.low[index] > df1.low[i]:
            is_low_pivot = False
        if df1.high[index] < df1.high[i]:
            is_high_pivot = False
    
    if is_low_pivot:
        return 1  # Low pivot
    elif is_high_pivot:
        return 2  # High pivot
    else:
        return 0  # No pivot

# Apply the pivotid function to the DataFrame
df['pivot'] = df.apply(lambda x: pivotid(df, x.name, 4), axis=1)


In [3]:
import numpy as np


def pointpos(row):
    """Add pointpos column to have a nicer visualization."""
    
    if row['pivot'] == 1:
        return row['low'] - 1e-3  
    elif row['pivot'] == 2:
        return row['high'] + 1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(pointpos, axis=1)


In [4]:
import plotly.graph_objects as go
dfpl = df[5050:5200]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.show()

In [5]:
from scipy.stats import linregress
import plotly.graph_objects as go

def detect_flag(candle, backcandles, window, plot_flag=False, pivot_count=3):
    """
    Detect if a candle is part of a flag pattern based on the last high and low pivot points
    and linear regression of them.
    """

    localdf = df[candle - backcandles - window:candle - window]
    highs = localdf[localdf['pivot'] == 2].high.tail(pivot_count).values
    idxhighs = localdf[localdf['pivot'] == 2].high.tail(pivot_count).index
    lows = localdf[localdf['pivot'] == 1].low.tail(pivot_count).values
    idxlows = localdf[localdf['pivot'] == 1].low.tail(pivot_count).index

    # Check if we have enough pivot points
    if len(highs) == pivot_count and len(lows) == pivot_count:
        # Check the ordering of pivots (low -> high -> low -> high)
        order_condition_first_low = all(
            [idxlows[i] < idxhighs[i] and (idxhighs[i] < idxlows[i+1] if i != pivot_count - 1 else True)
             for i in range(pivot_count)]
        )
        # Check the ordering of pivots (high -> low -> high -> low)
        order_condition_first_high = all(
            [idxlows[i] > idxhighs[i] and (idxlows[i] < idxhighs[i+1] if i != pivot_count - 1 else True)
             for i in range(pivot_count)]
        )
        order_condition = order_condition_first_low or order_condition_first_high

        slmin, intercept_min, rmin, _, _ = linregress(idxlows, lows)
        slmax, intercept_max, rmax, _, _ = linregress(idxhighs, highs)

        # Check if the flag pattern conditions are met
        if (order_condition
            and abs(rmax) >= 0.9
            and abs(rmin) >= 0.9
            and slmin >= 0.0
            and slmax <= 0.0):
            
            if plot_flag:
                fig = go.Figure(data=[go.Candlestick(
                    x=localdf.index,
                    open=localdf['open'],
                    high=localdf['high'],
                    low=localdf['low'],
                    close=localdf['close'],
                    name="Candlestick"
                )])

                fig.add_scatter(
                    x=localdf.index, 
                    y=localdf['pointpos'], 
                    mode="markers", 
                    marker=dict(size=10, color="MediumPurple"),
                    name="Pivot Points"
                )

                fig.add_trace(go.Scatter(
                    x=idxlows, 
                    y=slmin * idxlows + intercept_min, 
                    mode='lines', 
                    name='Min Slope'
                ))
                fig.add_trace(go.Scatter(
                    x=idxhighs, 
                    y=slmax * idxhighs + intercept_max, 
                    mode='lines', 
                    name='Max Slope'
                ))

                fig.update_layout(
                    xaxis_rangeslider_visible=False,
                    plot_bgcolor='white',
                    xaxis=dict(showgrid=True, gridcolor='white'),
                    yaxis=dict(showgrid=True, gridcolor='white')
                )
                fig.show()

            return 1  # Flag pattern detected

    return 0  # No flag pattern detected


In [6]:
detect_flag(5118, 35, 4, plot_flag=True) # check flag for a single candle

1

In [7]:
df['flag'] = df.index.map(lambda x: detect_flag(x, backcandles=35, window=4, pivot_count=3))
flagged_df = df[df['flag'] != 0]
flagged_df


,time,open,high,low,close,pivot,pointpos,flag
3265,18.09.2023 04:00:00.000000,1.06715,1.06738,1.06611,1.06621,2,1.06838,1
3266,18.09.2023 05:00:00.000000,1.06620,1.06683,1.06618,1.06655,0,NaN,1
3267,18.09.2023 06:00:00.000000,1.06651,1.06685,1.06633,1.06642,0,NaN,1
3366,22.09.2023 09:00:00.000000,1.06494,1.06599,1.06473,1.06597,0,NaN,1
3367,22.09.2023 10:00:00.000000,1.06597,1.06680,1.06149,1.06454,1,1.06049,1
5118,05.01.2024 08:00:00.000000,1.09374,1.09374,1.09245,1.09279,0,NaN,1
5119,05.01.2024 09:00:00.000000,1.09279,1.09336,1.09225,1.09234,0,NaN,1
5120,05.01.2024 10:00:00.000000,1.09233,1.09259,1.09034,1.09077,0,NaN,1
5121,05.01.2024 11:00:00.000000,1.09076,1.09306,1.09027,1.09128,0,NaN,1
5122,05.01.2024 12:00:00.000000,1.09128,1.09139,1.09037,1.09128,0,NaN,1
